based on:
[Python bokeh](https://bokeh.pydata.org/)
[ggplot2 reference](https://ggplot2.tidyverse.org/reference/)


In [29]:
import pandas as pd
from bokeh.plotting import figure, show
from math import atan, degrees
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6, Blues9
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource, Range1d, LinearColorMapper, ColorBar


mtcars = pd.read_csv('mtcars.csv', index_col=0)
mpg =  pd.read_csv('mpg.csv', index_col=0)
diamonds =  pd.read_csv('diamonds.csv', index_col=0)
output_notebook()

Loading BokehJS ...

In [34]:
#p <- ggplot(mtcars, aes(wt, mpg)) + geom_point()
def create_p():
    p = figure(title="My plot", plot_width=800, plot_height=400)
    p.circle(x='wt',y='mpg',source=mtcars)
    return p

#### Layer: geoms


#### geom_abline() geom_hline() geom_vline() 
Reference lines: horizontal, vertical, and diagonal. Unfortunately this functionality is not directly accessible in bokeh. The vline and hline functionality can be replicated with the ray glyph and special options. The ray will maintain its angle even after changing axis limits, so we should use the segment function instead.
Notes: The default angle_units are "rad" but can also be changed to "deg". To have an “infinite” ray, that always extends to the edge of the plot, specify 0 for the length.

In [35]:
p = create_p()
#p + geom_vline(xintercept = 5)
p.ray(x=5, y=0, length=0, angle=90, angle_units="deg")
show(p)

In [36]:
p = create_p()
#p + geom_vline(xintercept = 1:5)
p = figure(title="My plot", plot_width=800, plot_height=400)
p.circle(x='wt',y='mpg',source=mtcars)
p.ray(x=list(range(1,6)), y=[0]*5, length=0, angle=90, angle_units="deg")
show(p)

In [37]:
p = create_p()
#p + geom_hline(yintercept = 20)
p.ray(x=0, y=20, length=0, angle=0, angle_units="deg")
show(p)

In [38]:
p = create_p()
#p + geom_abline(intercept = 37, slope = -5)
def abline(xx, intercept, slope): return intercept + slope*xx
p.segment(x0 = -1, x1 = 10, y0 = abline(1,37,-5), y1 = abline(6,37,-5))
show(p)

#### geom_bar() geom_col() stat_count()
ggplot2 excels in plotting stats. For example, geom_bar() uses stat_count() by default while geom_col() uses stat_identity().
In bokeh, stats functions are not included yet, so the pandas Series functionality is used prior to plotting.  
Note: The bar `y` variable in ggplot2 is called `top` in bokeh. Also the bottom and width of the bars should be indicated. 

In [39]:
#g <- ggplot(mpg, aes(class))
mpg['counts'] = 0
mpg_plot = mpg[['class','counts']].groupby(by='class',as_index=False).count()
def create_gv():
    g = figure(title="My plot", plot_width=800, plot_height=400, x_range = mpg_plot['class'])
    return g
def create_gh():
    g = figure(title="My plot", plot_width=800, plot_height=400, y_range = mpg_plot['class'])
    return g


In [41]:
g = create_gv()
#g + geom_bar()
g.vbar(x='class', top='counts', width=0.9, bottom=0, source = mpg_plot)
show(g)

In [43]:
g = create_gv()
#g + geom_bar(aes(weight = displ))
mpg_plot = mpg[['class','displ']].groupby(by='class',as_index=False).sum()
g.vbar(x='class', top='displ', width=0.9, bottom=0, source = mpg_plot)
show(g)

In [44]:
from bokeh.core.properties import value
mycolors = [Spectral6[0],Spectral6[1],Spectral6[5]]
mpg_plot = mpg[['class','drv','counts']].groupby(by=['class','drv']).count().unstack(fill_value=0)
mylegend = [colname[1] for colname in mpg_plot.columns]
mpg_plot.columns = mylegend
mpg_plot['class'] = mpg_plot.index
g = create_gv()
#g + geom_bar(aes(fill = drv))
g.vbar_stack(mylegend, x='class', width=0.9, source = mpg_plot,
             color = mycolors,
             legend = [value(colname) for colname in mylegend])
g.legend.location = "top_left"
show(g)

In [45]:
g = create_gh()
#g +
# geom_bar(aes(fill = drv), position = position_stack(reverse = TRUE)) +
# coord_flip() +
# theme(legend.position = "top")
g.hbar_stack(mylegend, y='class', height=0.9, source = mpg_plot,
             color = mycolors,
             legend = [value(colname) for colname in mylegend])
g.legend.location = "bottom_right"
show(g)

#### geom_bin2d() stat_bin_2d() 
Again, ggplot2 excels in plotting stats. geom_bin2d divides the plane into rectangles, counts the number of cases in each rectangle. In bokeh, stats functions are not included yet, so the pandas Series functionality is used prior to plotting.  
Note: Fortunately hex_bin is available! 

In [46]:
#d <- ggplot(diamonds, aes(x, y)) + xlim(4, 10) + ylim(4, 10)
diamonds['x'] = pd.to_numeric(diamonds['x'])
diamonds['y'] = pd.to_numeric(diamonds['y'])
def create_d():
    d = figure(title="My plot", match_aspect=True)
    return d

In [33]:
Spectral6[0

TypeError: list indices must be integers or slices, not list

In [23]:
d = create_d()

x = [1,2,3,4,5,7,8,9,10]
y = [1,2,3,4,5,7,8,9,10]

#Use the field name of the column source
mapper = linear_cmap(field_name='count', palette=Spectral6 ,low=min(y) ,high=max(y))
mapper = LinearColorMapper(palette=Blues9, low=diamonds.count.min(), high=diamonds.count.max())

#d + geom_bin2d()
d.rect(x=diamonds['x'], y=diamonds['y'], width=1, height=1)
d.x_range=Range1d(4,10)
d.y_range=Range1d(4,10)
color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(d)

In [ ]:
#d <- ggplot(diamonds, aes(x, y)) + xlim(4, 10) + ylim(4, 10)
diamonds['carat'] = pd.to_numeric(diamonds['carat'])
diamonds['price'] = pd.to_numeric(diamonds['price'])
def create_d():
    d = figure(title="My plot", match_aspect=True)
    return d

In [47]:
d = create_d()
#d + geom_bin2d()
d.hexbin(x=diamonds['carat'], y=diamonds['price'], size=0.25)
show(d)

In [12]:
sum(diamonds['y'].isna())

0

In [ ]:
#geom_bar() geom_col() stat_count()
geom_abline() geom_hline() geom_vline() 


geom_bar() geom_col() stat_count() 
Bar charts

geom_bin2d() stat_bin_2d() 
Heatmap of 2d bin counts

geom_blank() 
Draw nothing

geom_boxplot() stat_boxplot() 
A box and whiskers plot (in the style of Tukey)

geom_contour() stat_contour() 
2d contours of a 3d surface

geom_count() stat_sum() 
Count overlapping points

geom_density() stat_density() 
Smoothed density estimates

geom_density_2d() stat_density_2d() 
Contours of a 2d density estimate

geom_dotplot() 
Dot plot

geom_errorbarh() 
Horizontal error bars

geom_hex() stat_bin_hex() 
Hexagonal heatmap of 2d bin counts

geom_freqpoly() geom_histogram() stat_bin() 
Histograms and frequency polygons

geom_jitter() 
Jittered points

geom_crossbar() geom_errorbar() geom_linerange() geom_pointrange() 
Vertical intervals: lines, crossbars & errorbars

geom_map() 
Polygons from a reference map

geom_path() geom_line() geom_step() 
Connect observations

geom_point() 
Points

geom_polygon() 
Polygons

geom_qq_line() stat_qq_line() geom_qq() stat_qq() 
A quantile-quantile plot

geom_quantile() stat_quantile() 
Quantile regression

geom_ribbon() geom_area() 
Ribbons and area plots

geom_rug() 
Rug plots in the margins

geom_segment() geom_curve() 
Line segments and curves

geom_smooth() stat_smooth() 
Smoothed conditional means

geom_spoke() 
Line segments parameterised by location, direction and distance

geom_label() geom_text() 
Text

geom_raster() geom_rect() geom_tile() 
Rectangles

geom_violin() stat_ydensity() 
Violin plot

stat_sf() geom_sf() geom_sf_label() geom_sf_text() coord_sf() 
Visualise sf objects